# Package Installation and import 📥
Here, we will install and import necessary packages for our code. 
These contain extra functions that we will need to make our program work

In [ ]:
import os
HOME = os.getcwd() # Get the current working directory
import ultralytics
from ultralytics import YOLO # Import YOLO class. This class is used to create a YOLOv8 model
from IPython.display import display, Image
from roboflow import Roboflow
import torch
from tqdm import tqdm

_______________________________________________________________________________________________

# Loading the dataset 🌐
We will first create the place where we will store the dataset.
Then, we will import a pre-existing dataset from the web using *Roboflow*. You can later try to use a different dataset by going onto https://universe.roboflow.com/ and finding another dataset. Then, change the **datasetname** variable


In [ ]:
#create the directory for the datasets:
datasets_dir = os.path.join(HOME, "datasets_yolo")
os.makedirs(datasets_dir, exist_ok=True)
os.chdir(datasets_dir)

# Download the dataset from Roboflow
datasetname = "dog_cat-test"
rf = Roboflow(api_key="f5pOg5frjQ3XAImBbLxX")
project = rf.workspace("ravensburgweingarten").project(datasetname)
dataset = project.version(1).download("yolov8")

## Fix data.yaml Don't forget to save!!! 🛠️
We need to manually fix this file because the dataset is intended to be used in a different environment.

1. in datasets_yolo/Dog_Cat-Test-1 <br>
2. go to data.yaml <br>
3. change 2 parameter as follow: <br>
     train: train/images <br>
     val: valid/images <br>

# The Dataset 🌐
Get acquainted with your dataset. Let us try looking at one element from it:

In [ ]:
#get the first element from the dataset (code on computer)
sample = f"{HOME}/datasets_yolo/{datasetname}/test/images/2T7OkT9RmTY_jpg.rf.7f68fbcbf3cd762eeefb2c42526c7807.jpg"
#get the full address of elements in the dataset
datasetname = "Dog_Cat-Test-1"
images = [os.path.join(datasets_dir, datasetname, "test", "images", image) for image in os.listdir(os.path.join(datasets_dir, datasetname, "test", "images"))]
labels = [os.path.join(datasets_dir, datasetname, "test", "labels", label) for label in os.listdir(os.path.join(datasets_dir, datasetname, "test", "labels"))]
#display five random pictures alongside their labels
for i in range(5):
    display(Image(filename=images[i]))
    print(labels[i])


_______________________________________________________________________________________________

# Training the model 👨‍🏫
We will now simply feed the dataset to YOLO so that it learns how to recognise elements from it. 

In [ ]:
%cd {HOME}

data_path= f"{dataset.location}/data.yaml"
model = YOLO("yolov8n.yaml")
results = model.train(data= data_path, epochs=5, imgsz=160, plots=True)

_______________________________________________________________________________________________

# Model Evaluation 📊
When we are analysing how well YOLO is at predicting the contents of an image, there are several metrics we can use.
The most important ones are the **training loss** and the **validation loss**. The lower these values are, the better your algorithm is at predicting data. 

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

# Furthermore, here is the F-1 Curve 📈
The F-1 curve tells us the overall performance of our model. It is particularly insightful because it **accounts for underrepresented classes**.
Imagine you have a thousand pictures of dogs and five of cats. You might have high accuracy if you always output dogs, but your F1 score will reflect this issue. 

In [ ]:
Image(filename=f'{HOME}/runs/detect/train/F1_curve.png', width=600)

_______________________________________________________________________________________________

## Testing the model ✍️
Previously, the model only saw pictures in the **train** folder. Now, we will show it the pictures in the **test** folder, pictures the model has never seen before. Based on how good the model's performance is with the test images, we can have an idea of what the model's performance with data in the real world will be.

## Here is the image we want to test 😽

In [ ]:
test1 =  f"{HOME}/datasets_yolo/Dog_Cat-Test-1/test/images/0262-jo9XwI6B8Rs_jpg.rf.7fb55bdcd52d24ffd635e25d8aa7e4ea.jpg"
Image(test1)

## Test pretrain model 📝
this is the model provided by Yolo (already train by the company)

In [ ]:
# Load a pretrained YOLOv8n model
%cd {HOME}
pretrainmodel = YOLO('yolov8n.pt')

# Run inference on an image
results_1 = pretrainmodel(test1)  # list of 1 Results object
for result in results_1:
    result.show()  # display to screen

## Test our model 📝

In [ ]:
# Load a model
%cd {HOME}
model_path=f"{HOME}/runs/detect/train/weights/best.pt"

ourmodel = YOLO(model_path)  # pretrained YOLOv8n model

# Run batched inference on a list of images
results_2 = ourmodel(test1)  # list of 1 Results object

# Process results list
for result in results_2:
    result.show()  # display to screen
   

## Can yolo detect everything? let's find out 🕵🏾

Here we want to detect spanner

In [ ]:
testimage = f"{HOME}/datasets_GD/hardware.jpg"
Image(testimage)

Can it detect??

In [ ]:
results_3 = pretrainmodel(testimage)  # list of 1 Results object
for result in results_3:
    result.show()  # display to screen

## YOLO cannot detect Spanner, WHY!!!! 😲

## Yolo is called 'Closed Set Object Detection', which has limitation on detection.<br>

This is all the objects that YOLO can detect <br>

In [ ]:
pretrainmodel.names

## So, how can we detect spanner??!! 🤔

## Let's find out in this workshop